In [ ]:
%pip install pandas
%pip install numpy
%pip install python-pptx

In [14]:
import pandas as pd
import numpy as np
from pptx import Presentation
from pptx.enum.shapes import MSO_SHAPE
from pptx.util import Inches
from pptx.oxml.ns import nsdecls
from pptx.oxml import parse_xml

In [15]:
## CONSTANTS
PRESENTATION_PATH = "awards_template.pptx"
DATASET = "final_data.csv"

COLUMNS_TO_DROP = ["Place", "Total Points"]

# Title slide
TITLE = "Science Olympiad at Cornell Division C"
SUBTITLE = "Awards Ceremony - DATE"

# Event transition slide
EVENT = "EVENT AWARDS"

# Spirit award winner
SPIRIT_AWARD_WINNER = "SCHOOL NAME"
SPIRIT_AWARD_SUBTITLE = "Best School Spirit Award Recipient"

# School transition slide
SCHOOL = "SCHOOL AWARDS"

In [16]:
presentation = Presentation(PRESENTATION_PATH)
slide_layouts = presentation.slide_layouts

In [17]:
# Create the title slide
title_slide = presentation.slides.add_slide(slide_layouts[0])
title_slide_title = title_slide.shapes.title
title_slide_title.text = TITLE

title_slide_subtitle = title_slide.placeholders[10]
title_slide_subtitle.text = SUBTITLE

In [18]:
# Create the event slide
event_title_slide = presentation.slides.add_slide(slide_layouts[1])
event_title_slide_title = event_title_slide.shapes.title
event_title_slide_title.text = EVENT

In [19]:
# Read in the dataset
df = pd.read_csv(DATASET, header=0, index_col=0)
df.index.name = None
df.columns.name = None

df.index = df.index.str.split(' \(').str[0]

# Keep the Place and Total columns only
df_school_awards = df[["Place", "Total Points"]]

# Drop the Place and Total columns
df_event_awards = df.drop(columns=COLUMNS_TO_DROP)

In [20]:
def create_column_slides(df):
    for column in df.columns:
        # Sort the dataframe by the column in ascending order
        sorted_df = df.sort_values(by=column, ascending=True)
        
        # Get the top 6 teams
        top_teams = sorted_df.head(6)

        # Because we can't transition between team names, we need to create a new slide for each, progressively revealing the top teams
        for i in range(7):
            column_slide = presentation.slides.add_slide(slide_layouts[2])
            column_slide_title = column_slide.shapes.title
            column_slide_title.text = column
            
            # Set the placeholders dynamically
            for j in range(i):
                column_slide.placeholders[15 - j].text = top_teams.index[5 - j]

create_column_slides(df_event_awards)

In [21]:
# Create the spirit slide
spirit_slide_holder = presentation.slides.add_slide(slide_layouts[4])
spirit_slide = presentation.slides.add_slide(slide_layouts[4])
spirit_slide.placeholders[15].text = SPIRIT_AWARD_WINNER
spirit_slide.placeholders[16].text = SPIRIT_AWARD_SUBTITLE

In [22]:
# Create the school slide
event_title_slide = presentation.slides.add_slide(slide_layouts[1])
event_title_slide_title = event_title_slide.shapes.title
event_title_slide_title.text = SCHOOL

In [ ]:
def create_school_awards(df, allow_same_school = False):
    if not allow_same_school:
        # This assumes that the index is formatted as "C01 - School Name." Modify as needed.
        df.index = df.index.str.split(' - ').str[1].str.split(',').str[0]
        df = df[~df.index.duplicated(keep='first')]

    # Because we can't transition between team names, we need to create a new slide for each, progressively revealing the top teams
    for i in range(7):
        school_slide = presentation.slides.add_slide(slide_layouts[3])
        school_slide_title = school_slide.shapes.title
        school_slide_title.text = SCHOOL
        
        # Set the placeholders dynamically
        for j in range(i):
            school_slide.placeholders[15 - j].text = df.index[5 - j]

In [24]:
create_school_awards(df_school_awards)

In [25]:
# save pptx
presentation.save(PRESENTATION_PATH)